In [2]:
#Multi-Integer Linear Programming

!pip install pulp -q
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, LpStatus

# Setting parameters
num_groups = 5  # number of household groups

q_incentive = [180, 3000, 132, 102, 78] # per-household incentive offered to each group (qᵢ)

n_min = [200,  1,   100,  300,  0] # lower / upper bounds on the # of households that can be incentivised in each group
n_max = [400,  90,  148, 2374,  56]

B_budget = 100_000 # total budget ($)
G_min    = 3   # min # of groups to incentivize
N_min = 500    # min # of households to incentivize

# Decision variables
  # xᵢ = 1 if group i is selected for incentives, 0 otherwise
x = [LpVariable(f"x_{i}", cat="Binary") for i in range(num_groups)]

  # nᵢ = # households incentivised in group i (integer, ≥0)
n = [LpVariable(f"n_{i}", lowBound=0, upBound=n_max[i], cat="Integer") for i in range(num_groups)]

# Model
model = LpProblem("Incentive_Cost_Minimisation", LpMinimize)

total_spend = lpSum(q_incentive[i] * n[i] for i in range(num_groups))
model += total_spend, "Total_cost"

# Constraints
  # Total budget
total_spend = lpSum(q_incentive[i] * n[i] for i in range(num_groups))

model += total_spend <= B_budget,        "Budget_upper"
model += total_spend >= 0.90 * B_budget, "Budget_lower"

  # minimum number of total households
model += lpSum(n[i] for i in range(num_groups)) >= N_min
  # minimum number of total groups
model += lpSum(x[i] for i in range(num_groups)) >= G_min, "Min_groups"

  # big-M formulation
for i in range(num_groups):
    model += n[i] >= n_min[i] * x[i],             f"min_size_group_{i}"
    model += n[i] <= n_max[i] * x[i],             f"max_size_group_{i}"

  # Priority constraints on picking groups (2>0>3>1>4)
model += x[2] == 1, "force_group_2"
model += x[0] <= x[2], "prio_2_before_0"   # pick 0 only if 2 is picked
model += x[3] <= x[0], "prio_0_before_3"   # pick 3 only if 0 is picked
model += x[1] <= x[3], "prio_3_before_1"   # pick 1 only if 3 is picked
model += x[4] <= x[1], "prio_1_before_4"   # pick 4 only if 1 is picked

# Results
model.solve()

print(f"Status        : {LpStatus[model.status]}")
print(f"Total payout  : ${model.objective.value():,.0f}\n")

for i in range(num_groups):
    print(f"Group {i}: selected={int(x[i].value())}  "
          f"households={int(n[i].value())}  "
          f"cost=${q_incentive[i]*n[i].value():,.0f}")

Status        : Optimal
Total payout  : $90,000

Group 0: selected=1  households=203  cost=$36,540
Group 1: selected=1  households=3  cost=$9,000
Group 2: selected=1  households=105  cost=$13,860
Group 3: selected=1  households=300  cost=$30,600
Group 4: selected=1  households=0  cost=$0
